In [1]:
from ib_insync import *

import datetime
from datetime import timedelta
import time
import nest_asyncio
import asyncio
import pandas as pd
import pytz
from dataclasses import dataclass

nest_asyncio.apply()
ib = IB()

RK_Max_CC_Loss_Cnt = 3;
RK_Day_Loss_Limit = 200;
RK_SYMBOL_LIST = 



gate_list = []




print_df_detail_log = False
print_detail_log = False;

@dataclass
class TradeControl:
    target_date: datetime.date
    trading_class: str
    is_allowed: bool = True  # 初始允许交易
    reason: str = ""

In [2]:
# ------------------------------------------------------------------------------------------------------------------------------------

def probe_trades(trades,symbol,target_date):

    df = util.df(trades)
    print(f"Target NY Date:{target_date}")

    # 提取属性到主表
    df['symbol'] = df['contract'].apply(lambda x: x.symbol)
    df['time'] = pd.to_datetime(df['execution'].apply(lambda x: x.time))
    df['cn_time'] = (df['time']
                      .dt.tz_convert('Asia/Shanghai')
                      .dt.tz_localize(None))
    df['ny_time'] = (df['time']
                 .dt.tz_convert('US/Eastern')
                 .dt.tz_localize(None))
    df['shares'] = df['execution'].apply(lambda x: x.shares)
    df['pnl'] = df['commissionReport'].apply(lambda x: x.realizedPNL)
    df['currency'] = df['contract'].apply(lambda x: x.currency)
    
    # 执行多条件过滤
    # 条件：tradingClass 是 MES 且 盈亏不为 0
    final_df = df[(df['symbol'] == symbol) & (df['pnl'] != 0) & (df['ny_time'].dt.date >= target_date)].copy()
    
    # 按照时间倒序排列（最近的在最上面）
    final_df = final_df.sort_values(by='ny_time', ascending=False).reset_index(drop=True)
        
    if print_df_detail_log:
        for i in range(1, 3, 1):
            print(df['contract'].iloc[i])
            # contract_df = util.df(list(df['contract']))
            print(df['execution'].iloc[i])
            # execution_df = util.df(list(df['execution']))
            print(df['commissionReport'].iloc[i])
            # commission_df = util.df(list(df['commissionReport']))
    
            # # 提取第一行的 commissionReport 对象
            # report = df.at[0, 'commissionReport']
            # print(report)
            print("----------------------------------------")
        print(final_df[['symbol', 'cn_time','ny_time', 'shares', 'pnl', 'currency']])
    return final_df[['symbol', 'cn_time','ny_time', 'shares', 'pnl', 'currency']]

# ------------------------------------------------------------------------------------------------------------------------------------

def check_risk(symbol,target_date):
    global gate_list
    
    # 2. 检查 gate 是否存在
    current_gate = next((g for g in gate_list if g['date'] == target_date and g['symbol'] == symbol), None)

    # 3. 如果不存在，则进行初始化
    if current_gate is None:
        print(f"初始运行：为 {symbol} 创建日期 {target_date} 的 gate 对象")
        new_gate = {
            "date": target_date,
            "symbol": symbol,
            "is_allowed": True,
            "reason": ""
        }
        gate_list.append(new_gate)
        current_gate = new_gate

    # 4. 检查 gate 状态：是否禁止交易
    if not current_gate['is_allowed']:
        print(f"禁入拦截：{symbol} 的交易状态为 [禁止]。原因: {current_gate['reason']}")
        return current_gate

    # 1. 获取所有成交
    # target_date_str = '20260210'
    # filter = ExecutionFilter(time=f"{target_date_str}-00:00:00")
    # trades = ib.reqExecutions(filter)
    trades = ib.fills()

    if not trades:
        print("获取缓存数据，没有成交记录")
    
    # 过滤订单
    trades_df = probe_trades(trades, symbol, target_date)
    print("----------今日交易 Start----------")
    print(trades_df)
    print("----------今日交易 End----------")


    # 2. 执行风控规则
    # 2.1 DLL
    trade_dll = trades_df[trades_df['pnl'] < 0]['pnl'].sum()
    if trade_dll < RK_Day_Loss_Limit:
        current_gate['is_allowed'] = False
        current_gate['reason'] = f"Daily Loss Limit:{trade_dll},禁止交易"
        print(f"----------Daily Loss Limit:{trade_dll},\t禁止交易----------")

    # 2.2 连续亏损限制
    last_3_pnls = trades_df['pnl'].head(3).tolist()
    if all(p < 0 for p in last_3_pnls):
        current_gate['is_allowed'] = False
        current_gate['reason'] = f"Max Loss Cnt Limit:{3},禁止交易"
        print(f"----------Max Loss Cnt Limit:{3},\t\t禁止交易----------")
    return current_gate

In [3]:
class RiskControlManager:

    def __init__(self, ib_instance):
        
        self.ib = ib_instance
        self.processed_ids = set() # 核心：记录已处理过的订单，防止逻辑死循环

        # # 1. 彻底清空旧绑定
        # ib.newOrderEvent.clear()
        # ib.openOrderEvent.clear()
        # 2. 事件绑定
        # ------------------------------------------------------------------------------------------------------------------------------------
        # 事件名称	         触发时机
        # openOrderEvent	 最推荐： 订单状态有任何更新（包括初始化、修改、成交、撤单）。
        # newOrderEvent	     首次发现新订单时触发（对非 API 手工单触发较严格）。
        # orderStatusEvent	 仅当订单状态字段（Status）发生变化时触发。
        # execDetailsEvent	 当有成交（Fill）产生时触发，返回成交明细。
        # commissionReportEvent	成交后的佣金确认报告生成时触发。
        # cancelOrderEvent	订单被确认撤销时触发。

        self.ib.openOrderEvent   += self.onOpenOrder
        self.ib.orderStatusEvent += self.onOrderStatus
        self.ib.cancelOrderEvent += self.onCancelOrder

    def onNewOrder(self, trade):
        if print_detail_log:
            print(f"【订单新建】: {trade}")
        print("----------newOrderEvent----------")
        self.summary_log(trade)
    
    def onOpenOrder(self, trade):
        if print_detail_log:
            print(f"【订单开启】: {trade}")
        print("----------openOrderEvent----------")
        self.summary_log(trade)
         # 关键要素
        symbol = trade.contract.symbol
        p_id = trade.order.permId
        # local_time = trade.log[0].time.astimezone().strftime('%Y-%m-%d %H:%M:%S')
        orderStatus = trade.orderStatus.status

        # 1. 过滤：只在订单新出现且尚未成交/撤销时进行风控判定
        if p_id in self.processed_ids:
            return

        if orderStatus in ['PendingCancel', 'Cancelled', 'Filled', 'Inactive']:
            # print(f"ℹ️ 订单 {p_id} 处于终态或取消中 ({orderStatus})，跳过逻辑。")
            return

        # 2. 判定窗口：PreSubmitted（最理想）或 Submitted（挂单中）
        if orderStatus in ['PreSubmitted', 'Submitted']:
            # --- 运行风控规则判定 ---
            # 获取当前纽约时间
            now_ny = pd.Timestamp.now(tz='US/Eastern')
            # 减去x天并只保留日期部分
            target_date = (now_ny - timedelta(days=3)).date()  

            current_gate = check_risk(symbol,target_date)
            
            if not current_gate['is_allowed']:                
                print(f"🛑 [风控拦截]")
                print(f"   原因: {current_gate['reason']}")
                print(f"   订单: {trade.contract.symbol} {trade.order.action} {trade.order.totalQuantity}手")
                # 执行撤单 
                loop = asyncio.get_event_loop()
                loop.call_soon(self.safe_cancel, trade)
                # asyncio.create_task(self.safe_cancel(trade))
    
                # 标记为已处理，无论撤单是否成功，此单不再重复检查
                self.processed_ids.add(p_id)
            else:
                # 如果合规，也标记为已检查，避免下个状态更新（Submitted）时重复运行检查逻辑
                self.processed_ids.add(p_id)
                print(f"✅ [风控通过] 订单 {p_id} 准许入场")
        print("----------openOrderEvent End----------")
    
    def onOrderStatus(self, trade):
        if print_detail_log:
            print(f"【订单修改】: {trade}")
        print("----------orderStatusEvent----------")
        self.summary_log(trade)
        print("----------orderStatusEvent End----------")

    def onExecDetails(self, trade):
        if print_detail_log:
            print(f"【订单执行】: {trade}")
        print("----------execDetailsEvent----------")
        self.summary_log(trade)
        print("----------execDetailsEvent End----------")
    
    def onCommissionReport(self, trade):
        if print_detail_log:
            print(f"【订单盈亏】: {trade}")
        print("----------commissionReportEvent----------")
        self.summary_log(trade)
        print("----------commissionReportEvent End----------")
    
    def onCancelOrder(self, trade):
        if print_detail_log:
            print(f"【订单取消】: {trade}")
        print("----------cancelOrderEvent----------")
        self.summary_log(trade)
        print("----------cancelOrderEvent End----------")
        
    def safe_cancel(self, trade):
        # 不能勾选 
        # "使用负数来捆绑自动委托单"
        # "Use negative numbers to bind automatic orders"        
        trades = self.ib.reqOpenOrders()
        if print_detail_log:
            print("----------Download open orders start----------")
            print(trades)
            print("----------Download open orders end----------")

        # 在当前已同步的 trades 中寻找
        target = next((t for t in trades if t.order.permId == trade.order.permId and t.order.orderId !=0), None)
        if target:
            print(f"Target order: {target}")
            self.ib.cancelOrder(target.order)
            print("Order Canceled")
    
    def summary_log(self, trade):
        log_msg = (
            f"PermId: {trade.order.permId}, Symbol: {trade.contract.symbol},\n"
            f"action: {trade.order.action}, orderType: {trade.order.orderType}, orderStatus: {trade.orderStatus.status}, "
            f"filled: {trade.orderStatus.filled}, remaining: {trade.orderStatus.remaining}, avgFillPrice: {trade.orderStatus.avgFillPrice}"
        )
        print(log_msg)


In [4]:

# --- 绑定与连接 ---

try:
    ib.connect('127.0.0.1', 7497, clientId=0) # 连接 TWS 或 Gateway
    # 绑定监听
    ib.reqAutoOpenOrders(True)
    ib.reqAllOpenOrders()
    # --- 绑定与运行 ---
    risk_ctrl = RiskControlManager(ib)
    print("----------订单监听就绪----------")
    
    # ib.run()
    while ib.isConnected():
        # 核心：必须使用 ib.sleep()，它会处理后台产生的 newOrderEvent 回调
        ib.sleep(2)
except KeyboardInterrupt:
    print("----------检测到手动停止----------")
finally:
    print("----------订单监听结束----------")
    ib.disconnect()

----------订单监听就绪----------
----------openOrderEvent----------
PermId: 818802746, Symbol: MES,
action: SELL, orderType: LMT, orderStatus: PreSubmitted, filled: 0.0, remaining: 0.0, avgFillPrice: 0.0
初始运行：为 MES 创建日期 2026-02-13 的 gate 对象
Target NY Date:2026-02-13
----------今日交易 Start----------
   symbol             cn_time             ny_time  shares     pnl currency
0     MES 2026-02-13 23:51:49 2026-02-13 10:51:49     2.0 -119.98      USD
1     MES 2026-02-13 23:46:39 2026-02-13 10:46:39     2.0  -62.48      USD
2     MES 2026-02-13 23:43:51 2026-02-13 10:43:51     1.0  -37.49      USD
3     MES 2026-02-13 23:41:31 2026-02-13 10:41:31     1.0   -1.24      USD
4     MES 2026-02-13 23:38:40 2026-02-13 10:38:40     1.0  -14.99      USD
5     MES 2026-02-13 23:36:57 2026-02-13 10:36:57     1.0   48.76      USD
6     MES 2026-02-13 23:30:26 2026-02-13 10:30:26     1.0  -64.99      USD
7     MES 2026-02-13 23:18:52 2026-02-13 10:18:52     1.0    5.01      USD
8     MES 2026-02-13 23:06:56 202